In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_community.embeddings import GPT4AllEmbeddings

model_name = "nomic-embed-text-v1.f16.gguf"

gpt4all_kwargs = {'allow_download': 'True'}

embeddings  =  GPT4AllEmbeddings(
    model_name=model_name,
    gpt4all_kwargs=gpt4all_kwargs
)

In [3]:
WEAVIATE_URL = 'host.docker.internal'
index_name = "index_files"

import weaviate
weaviate_client = weaviate.connect_to_custom(
    http_host=WEAVIATE_URL,
    http_port=8080,
    http_secure=False,
    grpc_host=WEAVIATE_URL,
    grpc_port=50051,
    grpc_secure=False,
    # auth_credentials=AuthApiKey(weaviate_key),   # `weaviate_key`: your Weaviate API key
)

from langchain_weaviate.vectorstores import WeaviateVectorStore
vectorstore = WeaviateVectorStore(weaviate_client, index_name, "text", embedding=embeddings)

In [4]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.sqlite import SqliteSaver

In [5]:
model_name = "gpt-4o-mini"

In [6]:
memory = SqliteSaver.from_conn_string(":memory:")

In [7]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [8]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [9]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    vectorstore.as_retriever(),
    "vectorstore",
    "Searches and returns excerpts from pdf files indexed",
)
tools = [tool]

In [10]:
prompt = """You are a smart research assistant. Use the vectorstore to look up information if needed. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model=model_name)
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [11]:
messages = [HumanMessage(content="como faço para instalar o pje?")]

In [12]:
thread = {"configurable": {"thread_id": "1"}}

In [14]:
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_VIX4AZFneOhDRbpWEfd4CLSt', 'function': {'arguments': '{"query":"instalação PJe"}', 'name': 'vectorstore'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 128, 'total_tokens': 146}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_661538dc1f', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-e5df5a5b-cbb8-4db2-933d-b05e5067e660-0', tool_calls=[{'name': 'vectorstore', 'args': {'query': 'instalação PJe'}, 'id': 'call_VIX4AZFneOhDRbpWEfd4CLSt', 'type': 'tool_call'}], usage_metadata={'input_tokens': 128, 'output_tokens': 18, 'total_tokens': 146})]
Calling: {'name': 'vectorstore', 'args': {'query': 'instalação PJe'}, 'id': 'call_VIX4AZFneOhDRbpWEfd4CLSt', 'type': 'tool_call'}
Back to the model!
[ToolMessage(content='TRT04 - Roteiro de instalação do PJe Portable para uso do PJe\n\nVERSÃO DO PORTABLE DISPONIBILIZADA EM 03/12/2021: 1.13.10\n\nA con

In [16]:
messages = [HumanMessage(content="Você sabe qual o endereço do site mencionado")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_QWdVVOxRIAXf4T7MdtqKDyLC', 'function': {'arguments': '{"query":"site TRT4"}', 'name': 'vectorstore'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 1533, 'total_tokens': 1549}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_661538dc1f', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-265baa71-4097-4af2-a2f1-a21784fbf456-0', tool_calls=[{'name': 'vectorstore', 'args': {'query': 'site TRT4'}, 'id': 'call_QWdVVOxRIAXf4T7MdtqKDyLC', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1533, 'output_tokens': 16, 'total_tokens': 1549})]}
Calling: {'name': 'vectorstore', 'args': {'query': 'site TRT4'}, 'id': 'call_QWdVVOxRIAXf4T7MdtqKDyLC', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='Fonte: Secretaria de Tecnologia da Informação e Comunicações Última atualização: 07/03/2023 14:13\n\nhttps://www.tr

In [13]:
response = abot.graph.invoke({"messages": messages}, thread)
response

Calling: {'name': 'vectorstore', 'args': {'query': 'instalar PJe'}, 'id': 'call_FUj6unJqR0SgEN3AiSCExmMT', 'type': 'tool_call'}
Back to the model!


{'messages': [HumanMessage(content='como faço para instalar o pje?'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_FUj6unJqR0SgEN3AiSCExmMT', 'function': {'arguments': '{"query":"instalar PJe"}', 'name': 'vectorstore'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 130, 'total_tokens': 147}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_611b667b19', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-080ffa6d-ae18-41db-b233-24cda3bb4bab-0', tool_calls=[{'name': 'vectorstore', 'args': {'query': 'instalar PJe'}, 'id': 'call_FUj6unJqR0SgEN3AiSCExmMT', 'type': 'tool_call'}], usage_metadata={'input_tokens': 130, 'output_tokens': 17, 'total_tokens': 147}),
  ToolMessage(content='TRT04 - Roteiro de instalação do PJe Portable para uso do PJe\n\nVERSÃO DO PORTABLE DISPONIBILIZADA EM 03/12/2021: 1.13.10\n\nA configuração do computador para uso do PJe ficou muito mais simples, não sendo mais 